# 표본설계 버전


In [6]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from patsy import dmatrices

# 1. 데이터 로드
file_name = "D:/semi2/MH/ML/분석용.csv"
try:
    df = pd.read_csv(file_name, encoding='cp949')
except:
    df = pd.read_csv(file_name, encoding='utf-8-sig')

# 2. 분석 설정
target_col = 'clinical_DB' # 당뇨
blocks = {
    'Model 1': ['age', 'C(sex)', 'C(edu)', 'C(incm)', 'C(occp)', 'C(marry)'],
    'Model 2': ['C(FH_HE)', 'C(FH_DY)', 'C(FH_HAA)', 'C(FH_ST)', 'C(FH_DB)', 'C(FH_OS)'],
    'Model 3': ['C(HE_obe)','HE_ht', 'HE_wt', 'HE_BMI'],
    'Model 4': ['C(smoking)', 'C(alcohol)', 'pa_aerobic', 'sleep_time_wy', 'sleep_time_wd'],
    'Model 5': ['mh_PHQ_S', 'mh_GAD_S', 'C(BP1)', 'EQ5D']
}

def get_p_display(p):
    if p < 0.001: return f"{p:.3f} (***)"
    elif p < 0.01: return f"{p:.3f} (**)"
    elif p < 0.05: return f"{p:.3f} (*)"
    else: return f"{p:.3f}"

# 모든 결과를 임시 저장할 리스트
temp_results = []
cumulative_features = []

# 3. 모델별 분석 실행 (핵심 분석 로직 유지)
for i in range(1, 6):
    model_name = f'Model {i}'
    cumulative_features.extend(blocks[model_name])
    
    clean_vars = [v.split('(')[-1].split(')')[0].split(',')[0] for v in cumulative_features]
    temp_df = df.dropna(subset=[target_col, 'wt_itvex', 'psu'] + clean_vars).copy()
    
    formula = f"{target_col} ~ {' + '.join(cumulative_features)}"
    y, X = dmatrices(formula, data=temp_df, return_type='dataframe')
    
    model = sm.GLM(y, X, family=sm.families.Binomial(), var_weights=temp_df['wt_itvex'])
    
    try:
        result = model.fit(cov_type='cluster', cov_kwds={'groups': temp_df['psu']})
        params = result.params
        conf = result.conf_int()
        pvals = result.pvalues
        
        for var in X.columns:
            if var == 'Intercept': continue
            
            if '[T.' in var:
                v_main = var.split('[')[0].replace('C(', '').replace(')', '')
                category = var.split('[T.')[1].replace(']', '')
            else:
                v_main = var
                category = 'Continuous'
            
            or_val = np.exp(params[var])
            ci_low, ci_high = np.exp(conf.loc[var])
            
            temp_results.append({
                '변수명': v_main,
                '카테고리': category,
                '모델': model_name,
                'OR (95% CI)': f"{or_val:.2f} ({ci_low:.2f} - {ci_high:.2f})",
                'p-value': get_p_display(pvals[var])
            })
    except Exception as e:
        print(f"{model_name} 분석 중 오류 발생: {e}")

# 4. 수직 정렬 로직 (변수명 -> 카테고리 -> 모델 1,2,3,4,5 순서)
df_res = pd.DataFrame(temp_results)
final_df_list = []

# 분석에 들어간 모든 변수명 순서대로 추출 (중복 제거)
unique_vars = []
for block in blocks.values():
    for v in block:
        clean_v = v.split('(')[-1].split(')')[0].split(',')[0]
        if clean_v not in unique_vars:
            unique_vars.append(clean_v)

for v in unique_vars:
    v_data = df_res[df_res['변수명'] == v]
    if v_data.empty: continue
    
    # 해당 변수의 카테고리들 (예: edu의 1, 2, 3)
    unique_cats = v_data['카테고리'].unique()
    
    for cat in unique_cats:
        cat_data = v_data[v_data['카테고리'] == cat]
        
        # 범주형인 경우 REF 행 추가 (변수와 카테고리가 처음 등장할 때 한 번만)
        if cat != 'Continuous':
            # REF는 모든 모델 공통이므로 맨 위에 한 줄만 추가
            final_df_list.append({
                '변수명': v, '카테고리': 'REF', '모델': '-', 'OR (95% CI)': 'REF', 'p-value': '-'
            })
        
        # [핵심] 해당 변수/카테고리에 대해 모델 1부터 5까지 수직으로 쌓기
        for i in range(1, 6):
            model_name = f'Model {i}'
            model_row = cat_data[cat_data['모델'] == model_name]
            
            if not model_row.empty:
                final_df_list.append(model_row.iloc[0].to_dict())

# 5. 저장
final_output = pd.DataFrame(final_df_list)
final_output.to_excel(f'당뇨(표본설계).xlsx', index=False)
print("수직 정렬 구조의 분석 표 저장이 완료되었습니다.")

c:\Users\asia\anaconda3\Lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1655: SpecificationWarning: cov_type not fully supported with var_weights
  warnings.warn('cov_type not fully supported with var_weights',
c:\Users\asia\anaconda3\Lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1655: SpecificationWarning: cov_type not fully supported with var_weights
  warnings.warn('cov_type not fully supported with var_weights',
c:\Users\asia\anaconda3\Lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1655: SpecificationWarning: cov_type not fully supported with var_weights
  warnings.warn('cov_type not fully supported with var_weights',
c:\Users\asia\anaconda3\Lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1655: SpecificationWarning: cov_type not fully supported with var_weights
  warnings.warn('cov_type not fully supported with var_weights',


수직 정렬 구조의 분석 표 저장이 완료되었습니다.


c:\Users\asia\anaconda3\Lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1655: SpecificationWarning: cov_type not fully supported with var_weights
  warnings.warn('cov_type not fully supported with var_weights',


In [7]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from patsy import dmatrices

# 1. 데이터 로드
file_name = "D:/semi2/MH/ML/분석용.csv"
try:
    df = pd.read_csv(file_name, encoding='cp949')
except:
    df = pd.read_csv(file_name, encoding='utf-8-sig')

# 2. 분석 설정
target_col = 'clinical_DY' # 이상지질혈증
blocks = {
    'Model 1': ['age', 'C(sex)', 'C(edu)', 'C(incm)', 'C(occp)', 'C(marry)'],
    'Model 2': ['C(FH_HE)', 'C(FH_DY)', 'C(FH_HAA)', 'C(FH_ST)', 'C(FH_DB)', 'C(FH_OS)'],
    'Model 3': ['C(HE_obe)','HE_ht', 'HE_wt', 'HE_BMI'],
    'Model 4': ['C(smoking)', 'C(alcohol)', 'pa_aerobic', 'sleep_time_wy', 'sleep_time_wd'],
    'Model 5': ['mh_PHQ_S', 'mh_GAD_S', 'C(BP1)', 'EQ5D']
}

def get_p_display(p):
    if p < 0.001: return f"{p:.3f} (***)"
    elif p < 0.01: return f"{p:.3f} (**)"
    elif p < 0.05: return f"{p:.3f} (*)"
    else: return f"{p:.3f}"

# 모든 결과를 임시 저장할 리스트
temp_results = []
cumulative_features = []

# 3. 모델별 분석 실행 (핵심 분석 로직 유지)
for i in range(1, 6):
    model_name = f'Model {i}'
    cumulative_features.extend(blocks[model_name])
    
    clean_vars = [v.split('(')[-1].split(')')[0].split(',')[0] for v in cumulative_features]
    temp_df = df.dropna(subset=[target_col, 'wt_itvex', 'psu'] + clean_vars).copy()
    
    formula = f"{target_col} ~ {' + '.join(cumulative_features)}"
    y, X = dmatrices(formula, data=temp_df, return_type='dataframe')
    
    model = sm.GLM(y, X, family=sm.families.Binomial(), var_weights=temp_df['wt_itvex'])
    
    try:
        result = model.fit(cov_type='cluster', cov_kwds={'groups': temp_df['psu']})
        params = result.params
        conf = result.conf_int()
        pvals = result.pvalues
        
        for var in X.columns:
            if var == 'Intercept': continue
            
            if '[T.' in var:
                v_main = var.split('[')[0].replace('C(', '').replace(')', '')
                category = var.split('[T.')[1].replace(']', '')
            else:
                v_main = var
                category = 'Continuous'
            
            or_val = np.exp(params[var])
            ci_low, ci_high = np.exp(conf.loc[var])
            
            temp_results.append({
                '변수명': v_main,
                '카테고리': category,
                '모델': model_name,
                'OR (95% CI)': f"{or_val:.2f} ({ci_low:.2f} - {ci_high:.2f})",
                'p-value': get_p_display(pvals[var])
            })
    except Exception as e:
        print(f"{model_name} 분석 중 오류 발생: {e}")

# 4. 수직 정렬 로직 (변수명 -> 카테고리 -> 모델 1,2,3,4,5 순서)
df_res = pd.DataFrame(temp_results)
final_df_list = []

# 분석에 들어간 모든 변수명 순서대로 추출 (중복 제거)
unique_vars = []
for block in blocks.values():
    for v in block:
        clean_v = v.split('(')[-1].split(')')[0].split(',')[0]
        if clean_v not in unique_vars:
            unique_vars.append(clean_v)

for v in unique_vars:
    v_data = df_res[df_res['변수명'] == v]
    if v_data.empty: continue
    
    # 해당 변수의 카테고리들 (예: edu의 1, 2, 3)
    unique_cats = v_data['카테고리'].unique()
    
    for cat in unique_cats:
        cat_data = v_data[v_data['카테고리'] == cat]
        
        # 범주형인 경우 REF 행 추가 (변수와 카테고리가 처음 등장할 때 한 번만)
        if cat != 'Continuous':
            # REF는 모든 모델 공통이므로 맨 위에 한 줄만 추가
            final_df_list.append({
                '변수명': v, '카테고리': 'REF', '모델': '-', 'OR (95% CI)': 'REF', 'p-value': '-'
            })
        
        # [핵심] 해당 변수/카테고리에 대해 모델 1부터 5까지 수직으로 쌓기
        for i in range(1, 6):
            model_name = f'Model {i}'
            model_row = cat_data[cat_data['모델'] == model_name]
            
            if not model_row.empty:
                final_df_list.append(model_row.iloc[0].to_dict())

# 5. 저장
final_output = pd.DataFrame(final_df_list)
final_output.to_excel(f'이상지질혈증(표본설계).xlsx', index=False)
print("수직 정렬 구조의 분석 표 저장이 완료되었습니다.")

c:\Users\asia\anaconda3\Lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1655: SpecificationWarning: cov_type not fully supported with var_weights
  warnings.warn('cov_type not fully supported with var_weights',
c:\Users\asia\anaconda3\Lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1655: SpecificationWarning: cov_type not fully supported with var_weights
  warnings.warn('cov_type not fully supported with var_weights',
c:\Users\asia\anaconda3\Lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1655: SpecificationWarning: cov_type not fully supported with var_weights
  warnings.warn('cov_type not fully supported with var_weights',
c:\Users\asia\anaconda3\Lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1655: SpecificationWarning: cov_type not fully supported with var_weights
  warnings.warn('cov_type not fully supported with var_weights',


수직 정렬 구조의 분석 표 저장이 완료되었습니다.


c:\Users\asia\anaconda3\Lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1655: SpecificationWarning: cov_type not fully supported with var_weights
  warnings.warn('cov_type not fully supported with var_weights',


# 표준화 추가 버전

In [8]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from patsy import dmatrices

# 1. 데이터 로드
file_name = "D:/semi2/MH/ML/분석용.csv"
try:
    df = pd.read_csv(file_name, encoding='cp949')
except:
    df = pd.read_csv(file_name, encoding='utf-8-sig')

# 2. 분석 설정
target_col = 'clinical_DB'
blocks = {
    'Model 1': ['age', 'C(sex)', 'C(edu)', 'C(incm)', 'C(occp)', 'C(marry)'],
    'Model 2': ['C(FH_HE)', 'C(FH_DY)', 'C(FH_HAA)', 'C(FH_ST)', 'C(FH_DB)', 'C(FH_OS)'],
    'Model 3': ['C(HE_obe)','HE_ht', 'HE_wt', 'HE_BMI'],
    'Model 4': ['C(smoking)', 'C(alcohol)', 'pa_aerobic', 'sleep_time_wy', 'sleep_time_wd'],
    'Model 5': ['mh_PHQ_S', 'mh_GAD_S', 'C(BP1)', 'EQ5D']
}

# [추가] 표준화(Standardization)가 필요한 변수 리스트
standardize_vars = ['mh_PHQ_S', 'mh_GAD_S', 'EQ5D']

def get_p_display(p):
    if p < 0.001: return f"{p:.3f} (***)"
    elif p < 0.01: return f"{p:.3f} (**)"
    elif p < 0.05: return f"{p:.3f} (*)"
    else: return f"{p:.3f}"

temp_results = []
cumulative_features = []

# 3. 모델별 분석 실행
for i in range(1, 6):
    model_name = f'Model {i}'
    cumulative_features.extend(blocks[model_name])
    
    clean_vars = [v.split('(')[-1].split(')')[0].split(',')[0] for v in cumulative_features]
    temp_df = df.dropna(subset=[target_col, 'wt_itvex', 'psu'] + clean_vars).copy()
    
    # [핵심] 지정된 변수들에 대해 표준화 실행 (Z-score)
    for col in standardize_vars:
        if col in temp_df.columns and col in clean_vars:
            # (X - mean) / std 적용
            temp_df[col] = (temp_df[col] - temp_df[col].mean()) / temp_df[col].std()
    
    formula = f"{target_col} ~ {' + '.join(cumulative_features)}"
    y, X = dmatrices(formula, data=temp_df, return_type='dataframe')
    
    model = sm.GLM(y, X, family=sm.families.Binomial(), var_weights=temp_df['wt_itvex'])
    
    try:
        result = model.fit(cov_type='cluster', cov_kwds={'groups': temp_df['psu']})
        params = result.params
        conf = result.conf_int()
        pvals = result.pvalues
        
        for var in X.columns:
            if var == 'Intercept': continue
            
            if '[T.' in var:
                v_main = var.split('[')[0].replace('C(', '').replace(')', '')
                category = var.split('[T.')[1].replace(']', '')
            else:
                v_main = var
                category = 'Continuous (Standardized)' if v_main in standardize_vars else 'Continuous'
            
            or_val = np.exp(params[var])
            ci_low, ci_high = np.exp(conf.loc[var])
            
            temp_results.append({
                '변수명': v_main,
                '카테고리': category,
                '모델': model_name,
                'OR (95% CI)': f"{or_val:.2f} ({ci_low:.2f} - {ci_high:.2f})",
                'p-value': get_p_display(pvals[var])
            })
    except Exception as e:
        print(f"{model_name} 분석 중 오류 발생: {e}")

# 4. 수직 정렬 로직 (동일)
df_res = pd.DataFrame(temp_results)
final_df_list = []

unique_vars = []
for block in blocks.values():
    for v in block:
        clean_v = v.split('(')[-1].split(')')[0].split(',')[0]
        if clean_v not in unique_vars:
            unique_vars.append(clean_v)

for v in unique_vars:
    v_data = df_res[df_res['변수명'] == v]
    if v_data.empty: continue
    
    unique_cats = v_data['카테고리'].unique()
    for cat in unique_cats:
        cat_data = v_data[v_data['카테고리'] == cat]
        if 'Continuous' not in cat:
            final_df_list.append({
                '변수명': v, '카테고리': 'REF', '모델': '-', 'OR (95% CI)': 'REF', 'p-value': '-'
            })
        
        for i in range(1, 6):
            model_name = f'Model {i}'
            model_row = cat_data[cat_data['모델'] == model_name]
            if not model_row.empty:
                final_df_list.append(model_row.iloc[0].to_dict())

# 5. 저장
final_output = pd.DataFrame(final_df_list)
final_output.to_excel(f'당뇨(표본설계+표준화).xlsx', index=False)
print("표준화 및 수직 정렬이 적용된 분석이 완료되었습니다.")

c:\Users\asia\anaconda3\Lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1655: SpecificationWarning: cov_type not fully supported with var_weights
  warnings.warn('cov_type not fully supported with var_weights',
c:\Users\asia\anaconda3\Lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1655: SpecificationWarning: cov_type not fully supported with var_weights
  warnings.warn('cov_type not fully supported with var_weights',
c:\Users\asia\anaconda3\Lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1655: SpecificationWarning: cov_type not fully supported with var_weights
  warnings.warn('cov_type not fully supported with var_weights',
c:\Users\asia\anaconda3\Lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1655: SpecificationWarning: cov_type not fully supported with var_weights
  warnings.warn('cov_type not fully supported with var_weights',


표준화 및 수직 정렬이 적용된 분석이 완료되었습니다.


c:\Users\asia\anaconda3\Lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1655: SpecificationWarning: cov_type not fully supported with var_weights
  warnings.warn('cov_type not fully supported with var_weights',


In [9]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from patsy import dmatrices

# 1. 데이터 로드
file_name = "D:/semi2/MH/ML/분석용.csv"
try:
    df = pd.read_csv(file_name, encoding='cp949')
except:
    df = pd.read_csv(file_name, encoding='utf-8-sig')

# 2. 분석 설정
target_col = 'clinical_DY'
blocks = {
    'Model 1': ['age', 'C(sex)', 'C(edu)', 'C(incm)', 'C(occp)', 'C(marry)'],
    'Model 2': ['C(FH_HE)', 'C(FH_DY)', 'C(FH_HAA)', 'C(FH_ST)', 'C(FH_DB)', 'C(FH_OS)'],
    'Model 3': ['C(HE_obe)','HE_ht', 'HE_wt', 'HE_BMI'],
    'Model 4': ['C(smoking)', 'C(alcohol)', 'pa_aerobic', 'sleep_time_wy', 'sleep_time_wd'],
    'Model 5': ['mh_PHQ_S', 'mh_GAD_S', 'C(BP1)', 'EQ5D']
}

# [추가] 표준화(Standardization)가 필요한 변수 리스트
standardize_vars = ['mh_PHQ_S', 'mh_GAD_S', 'EQ5D']

def get_p_display(p):
    if p < 0.001: return f"{p:.3f} (***)"
    elif p < 0.01: return f"{p:.3f} (**)"
    elif p < 0.05: return f"{p:.3f} (*)"
    else: return f"{p:.3f}"

temp_results = []
cumulative_features = []

# 3. 모델별 분석 실행
for i in range(1, 6):
    model_name = f'Model {i}'
    cumulative_features.extend(blocks[model_name])
    
    clean_vars = [v.split('(')[-1].split(')')[0].split(',')[0] for v in cumulative_features]
    temp_df = df.dropna(subset=[target_col, 'wt_itvex', 'psu'] + clean_vars).copy()
    
    # [핵심] 지정된 변수들에 대해 표준화 실행 (Z-score)
    for col in standardize_vars:
        if col in temp_df.columns and col in clean_vars:
            # (X - mean) / std 적용
            temp_df[col] = (temp_df[col] - temp_df[col].mean()) / temp_df[col].std()
    
    formula = f"{target_col} ~ {' + '.join(cumulative_features)}"
    y, X = dmatrices(formula, data=temp_df, return_type='dataframe')
    
    model = sm.GLM(y, X, family=sm.families.Binomial(), var_weights=temp_df['wt_itvex'])
    
    try:
        result = model.fit(cov_type='cluster', cov_kwds={'groups': temp_df['psu']})
        params = result.params
        conf = result.conf_int()
        pvals = result.pvalues
        
        for var in X.columns:
            if var == 'Intercept': continue
            
            if '[T.' in var:
                v_main = var.split('[')[0].replace('C(', '').replace(')', '')
                category = var.split('[T.')[1].replace(']', '')
            else:
                v_main = var
                category = 'Continuous (Standardized)' if v_main in standardize_vars else 'Continuous'
            
            or_val = np.exp(params[var])
            ci_low, ci_high = np.exp(conf.loc[var])
            
            temp_results.append({
                '변수명': v_main,
                '카테고리': category,
                '모델': model_name,
                'OR (95% CI)': f"{or_val:.2f} ({ci_low:.2f} - {ci_high:.2f})",
                'p-value': get_p_display(pvals[var])
            })
    except Exception as e:
        print(f"{model_name} 분석 중 오류 발생: {e}")

# 4. 수직 정렬 로직 (동일)
df_res = pd.DataFrame(temp_results)
final_df_list = []

unique_vars = []
for block in blocks.values():
    for v in block:
        clean_v = v.split('(')[-1].split(')')[0].split(',')[0]
        if clean_v not in unique_vars:
            unique_vars.append(clean_v)

for v in unique_vars:
    v_data = df_res[df_res['변수명'] == v]
    if v_data.empty: continue
    
    unique_cats = v_data['카테고리'].unique()
    for cat in unique_cats:
        cat_data = v_data[v_data['카테고리'] == cat]
        if 'Continuous' not in cat:
            final_df_list.append({
                '변수명': v, '카테고리': 'REF', '모델': '-', 'OR (95% CI)': 'REF', 'p-value': '-'
            })
        
        for i in range(1, 6):
            model_name = f'Model {i}'
            model_row = cat_data[cat_data['모델'] == model_name]
            if not model_row.empty:
                final_df_list.append(model_row.iloc[0].to_dict())

# 5. 저장
final_output = pd.DataFrame(final_df_list)
final_output.to_excel(f'이상지질혈증(표본설계+표준화).xlsx', index=False)
print("표준화 및 수직 정렬이 적용된 분석이 완료되었습니다.")

c:\Users\asia\anaconda3\Lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1655: SpecificationWarning: cov_type not fully supported with var_weights
  warnings.warn('cov_type not fully supported with var_weights',
c:\Users\asia\anaconda3\Lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1655: SpecificationWarning: cov_type not fully supported with var_weights
  warnings.warn('cov_type not fully supported with var_weights',
c:\Users\asia\anaconda3\Lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1655: SpecificationWarning: cov_type not fully supported with var_weights
  warnings.warn('cov_type not fully supported with var_weights',
c:\Users\asia\anaconda3\Lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1655: SpecificationWarning: cov_type not fully supported with var_weights
  warnings.warn('cov_type not fully supported with var_weights',


표준화 및 수직 정렬이 적용된 분석이 완료되었습니다.


c:\Users\asia\anaconda3\Lib\site-packages\statsmodels\genmod\generalized_linear_model.py:1655: SpecificationWarning: cov_type not fully supported with var_weights
  warnings.warn('cov_type not fully supported with var_weights',
